In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

import cv2
import time

import sys
sys.path.insert(0, '..')

from web_interaction.main import open_game, start_game, wait_loading
from game.game import ProductOwnerGame
from environment.backlog_env import BacklogEnv
from environment.environment import ProductOwnerEnv
from environment.reward_sytem.base_reward_system import BaseRewardSystem
from environment.userstory_env import UserstoryEnv

from pipeline.study_agent import load_dqn_agent

from web_interaction import GameImageParser, GameCoordinator, WebController

from pet_logging import get_logger

from logging import Logger

In [2]:
def log_game_state(game: ProductOwnerGame, logger: Logger):
    logger.info(f'Sprint {game.context.current_sprint}')
    logger.info(f'Money {game.context.get_money()}')
    logger.info(f'Loyalty {game.context.get_loyalty()}')
    logger.info(f'Customers {game.context.customers}')
    logger.info(f'Credit {game.context.credit}')

In [3]:
logger = get_logger('WebInteraction')

image_parser = GameImageParser('../web_interaction/templates')
game_coordinator = GameCoordinator(image_parser)
game = ProductOwnerGame()

userstory_env = UserstoryEnv(4, 0, 0)
backlog_env = BacklogEnv(12, 0, 0, 0, 0, 0)
reward_system = BaseRewardSystem(config={})
env = ProductOwnerEnv(userstory_env, backlog_env, with_info=True, reward_system=reward_system)
env.game = game

web_controller = WebController(game_coordinator, logger)

In [4]:
agent = load_dqn_agent('../models/credit_start_model.pt')

c:\Users\kruto\OneDrive\Рабочий стол\forks\product_owner_rl\web_interaction\..\pipeline\study_agent.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent: DQN = torch.l

In [5]:
driver = open_game()

iframe = driver.find_element(by=By.ID, value='game_drop')

# open full screen
fullscreen_button = driver.find_element(by=By.CLASS_NAME, value="fullscreen_btn")
fullscreen_button.click()

height = iframe.rect["height"]  # 540 or 1028
width = iframe.rect["width"]  # 960 or 1920

wait_loading(iframe)

start_game(driver, iframe)

iframe.screenshot('game_state.png')

image = cv2.imread('game_state.png')

In [6]:
game_coordinator.skip_tutorial(game)
game_coordinator.insert_user_stories_from_image(game, image)

game.userstories.stories_list

[UserStoryCard(S, loyalty=0.04, customers=1.0]

In [7]:
game_coordinator.update_header_info(game, image)

log_game_state(game, logger)

2024-11-04 14:45:03,599 Sprint 4
2024-11-04 14:45:03,599 Money 33000.0
2024-11-04 14:45:03,600 Loyalty 4.0
2024-11-04 14:45:03,600 Customers 25.0
2024-11-04 14:45:03,601 Credit 300000


In [8]:
def apply_web_action(action: int, driver, iframe: WebElement, env: ProductOwnerEnv):
    if action == 0:  # start sprint
        web_controller.start_sprint(driver, iframe, env)
        return

    if action == 1:  # decompose
        web_controller.decompose(driver, iframe, env)
        return

    if action == 2:  # release
        web_controller.release_tasks(driver, iframe, env)
        return

    if action == 5:  # buy statistical research
        web_controller.buy_statistical_research(driver, iframe, env)
        return

    if action >= env.meta_action_dim:

        action -= env.meta_action_dim

    if action < env.userstory_env.max_action_num:
        web_controller.apply_user_story_action(action, driver, iframe, env)
        return

    action -= env.userstory_env.max_action_num

    if action < env.backlog_env.backlog_max_action_num:
        web_controller.apply_backlog_card_action(action, driver, iframe, env)
        return

    raise Exception(f"Unknown action: {action}")

In [9]:
while not game.context.done:
    state = env.recalculate_state()
    # print(state)

    info = env.get_info()
    # print(info)

    action = agent.get_action(state, info)
    logger.info(f'Action id: {action}')

    time.sleep(1)
    apply_web_action(action, driver, iframe, env)

    if env.game.context.current_sprint >= 35:
        logger.warning("Reached credit end!")
        break

2024-11-04 14:45:03,652 Action id: 7
2024-11-04 14:45:04,655 Start user story action: 0
2024-11-04 14:45:04,661 User story: UserStoryCard(S, loyalty=0.04, customers=1.0
2024-11-04 14:45:04,664 Found at position: (1556, 384)
2024-11-04 14:45:06,668 Reward: True
2024-11-04 14:45:06,670 Action id: 1
2024-11-04 14:45:07,672 Start decomposition
2024-11-04 14:45:10,663 Action id: 11
2024-11-04 14:45:11,665 Start moving backlog card
2024-11-04 14:45:13,034 Selected card Card(CardInfo(17, (255, 211, 143), 2708353180240, S, UserCardType.S))
2024-11-04 14:45:13,035 Found at position (1516, 384)
2024-11-04 14:45:13,361 Clicked on card
2024-11-04 14:45:13,364 Action id: 0
2024-11-04 14:45:14,365 Start new sprint
2024-11-04 14:45:18,357 Action id: 20
2024-11-04 14:45:19,358 Start moving backlog card
2024-11-04 14:45:20,715 Selected card Card(CardInfo(6, (255, 211, 143), 2708353180240, S, UserCardType.S))
2024-11-04 14:45:20,717 Found at position (1597, 384)
2024-11-04 14:45:21,055 Clicked on card
2

In [10]:
raise Exception('Break before empty sprinst')

Exception: Break before empty sprinst

In [11]:
log_game_state(game, logger)

2024-11-04 14:46:56,861 Sprint 11
2024-11-04 14:46:56,861 Money 26000.0
2024-11-04 14:46:56,862 Loyalty 4.02
2024-11-04 14:46:56,862 Customers 0.02582
2024-11-04 14:46:56,863 Credit 237000


In [ ]:
while not game.context.done:
    time.sleep(1)
    apply_web_action(0, driver, iframe, env)

In [ ]:
iframe.rect

{'height': 1028, 'width': 1920, 'x': 0, 'y': 0}

In [ ]:
raise Exception('Break before web driver exit')

Exception: Break before web driver exit

In [ ]:
game_coordinator.backlog_cards

[BacklogCardImageInfo((54, 79, 234), 17, (1517, 384)),
 BacklogCardImageInfo((54, 79, 234), 9, (1598, 384)),
 BacklogCardImageInfo((54, 79, 234), 7, (1517, 471)),
 BacklogCardImageInfo((54, 79, 234), 5, (1598, 471)),
 BacklogCardImageInfo((84, 36, 195), 15, (1517, 558)),
 BacklogCardImageInfo((84, 36, 195), 11, (1598, 558)),
 BacklogCardImageInfo((84, 36, 195), 8, (1517, 645)),
 BacklogCardImageInfo((84, 36, 195), 4, (1598, 645))]

In [ ]:
game.userstories.release

[UserStoryCard(S, loyalty=0.065, customers=1.0,
 UserStoryCard(S, loyalty=0.06, customers=1.0]

In [ ]:
for us in game.userstories.release:
    print(us.info.color)

(54, 79, 234)
(120, 79, 240)


In [ ]:
game_coordinator.user_stories

[UserStoryImageInfo((120, 79, 240), 0.05, 2.0, (1556, 384)),
 UserStoryImageInfo((255, 211, 143), 0.06, 2.0, (1556, 471))]

In [ ]:
image_parser.templates = image_parser._load_templates()

In [ ]:
image = cv2.imread('game_state.png')

game_coordinator.update_header_info(game, image)

In [ ]:
info

{'actions': [0, 3, 4, 5, 6, 7, 8, 9, 10]}

In [ ]:
state

array([3.40e+01, 3.79e+00, 1.54e-02, 2.82e+00, 3.00e-01, 2.00e+00,
       1.00e+00, 0.00e+00, 0.00e+00, 1.30e+01, 1.00e+00, 0.00e+00,
       0.00e+00, 1.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e

In [ ]:
env.game.context.current_sprint

11

In [ ]:
driver.quit()